url txt 파일 csv로 저장

In [21]:
import urllib.request
import csv

url = "https://raw.githubusercontent.com/laxmimerit/Human-Activity-Recognition-Using-Accelerometer-Data-and-CNN/master/WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt" # txt 파일의 URI
filename = "../data/human_activity.csv" # 저장할 파일 이름
column_names = ['user', 'activity', 'time', 'x', 'y', 'z']

# URI에서 파일 다운로드
urllib.request.urlretrieve(url, "temp.txt")

# txt 파일을 csv 파일로 변환하여 저장
with open("temp.txt", "r") as infile, open(filename, "w", newline="") as outfile:
    reader = csv.reader(infile, delimiter=",")
    writer = csv.writer(outfile, delimiter=",")

    # 열 이름 추가
    writer.writerow(column_names)

 # 데이터 행 추가
    for row in reader:
        try:
            row[5] = row[5].replace(";", "")
            writer.writerow(row)
        except IndexError:
            pass

print("변환 완료: " + filename)

# 임시로 다운로드한 txt 파일 삭제
import os
os.remove("temp.txt")


변환 완료: ../data/human_activity.csv


In [22]:
import pandas as pd
def normalize_dataframe(df):
    # x, y, z 열의 값을 -1과 1 사이 값으로 정규화
    df['x'] = 2 * (df['x'] - df['x'].min()) / (df['x'].max() - df['x'].min()) - 1
    df['y'] = 2 * (df['y'] - df['y'].min()) / (df['y'].max() - df['y'].min()) - 1
    df['z'] = 2 * (df['z'] - df['z'].min()) / (df['z'].max() - df['z'].min()) - 1

    # 정규화된 데이터프레임의 헤드값 출력
    print(df.head())

df_other = pd.read_csv('../data/df_other.csv')
# df_SisFall = pd.read_csv('../data/df_SisFall.csv')

In [23]:
df_other = pd.read_csv(filename, usecols=range(6))
df_other = df_other.iloc[:, [3, 4, 5, 1]]
normalize_dataframe(df_other)
df_other.head()

          x         y         z activity
0 -0.043713  0.628779  0.030396  Jogging
1  0.244807  0.557328  0.053206  Jogging
2  0.239299  0.538091  0.000674  Jogging
3 -0.039581  0.922140  0.158270  Jogging
4 -0.068502  0.599924  0.370473  Jogging


,x,y,z,activity
0,-0.043713,0.628779,0.030396,Jogging
1,0.244807,0.557328,0.053206,Jogging
2,0.239299,0.538091,0.000674,Jogging
3,-0.039581,0.922140,0.158270,Jogging
4,-0.068502,0.599924,0.370473,Jogging


In [24]:
df_SisFall = pd.read_csv("../data/SisFall_dataset.csv", usecols=[3, 4, 5, 9], names=["x", "y", "z", "activity"], skiprows=[0])

df_SisFall['x'] = df_SisFall['x'].astype(int)
df_SisFall['y'] = df_SisFall['y'].astype(int)
df_SisFall['z'] = df_SisFall['z'].astype(int)

normalize_dataframe(df_SisFall)
# df_SisFall.info()

          x         y         z activity
0 -0.005994 -0.010220 -0.067430     Fall
1 -0.006055 -0.010220 -0.067430     Fall
2 -0.006055 -0.010284 -0.067430     Fall
3 -0.006086 -0.010220 -0.067515     Fall
4 -0.006086 -0.010220 -0.067430     Fall


In [10]:
# import glob
#
# # Use glob to find all CSV files that match a pattern
# files = glob.glob('../glob/*.csv')
#
# # Create an empty list to store the DataFrames
# dfs = []
#
# # Loop over all CSV files and load them into DataFrames
# for file in files:
#     dfs.append(pd.read_csv(file))
#
# # Concatenate all DataFrames into a single DataFrame
# merged_df = pd.concat(dfs, ignore_index=True)
#
# # Write the merged DataFrame to a new CSV file
# merged_df.to_csv('../glob/merged.csv', index=False)

In [36]:
# df_other와 df_SisFall을 행 방향으로 합침
df_concat = pd.concat([df_SisFall, df_other])

# nan 값 제거
df_concat = df_concat.dropna()

# # csv 파일로 저장
# df_concat.to_csv("../data/df_concat_new.csv", index=False)


In [37]:
from sklearn.model_selection import train_test_split

# 독립변수, 종속변수 지정
X = df_concat[['x', 'y', 'z']]
y = df_concat['activity']

In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 레이블 인코딩
le = LabelEncoder()
df_concat["activity"] = le.fit_transform(df_concat["activity"])

# 표준화
scaler = StandardScaler()
df_concat[["x", "y", "z"]] = scaler.fit_transform(df_concat[["x", "y", "z"]])

# 학습 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 랜덤 포레스트 모델 학습
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# 테스트용 데이터로 예측
y_pred = rf.predict(X_test)

# 정확도 출력
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.6394201580211265


In [39]:
from sklearn.linear_model import LogisticRegression

# 로지스틱 회귀 모델 정의 및 학습
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train, y_train)

# 테스트 데이터로 예측 수행
y_pred = logreg.predict(X_test)

# 정확도 출력
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.4876


thisistrash